In [23]:
import numpy as np
from collections import deque
import numpy as np
import time

In [24]:
def matrix_to_tuple(matrix):
    return (matrix[0][0], matrix[0][1], matrix[0][2],
            matrix[1][0], matrix[1][1], matrix[1][2],
            matrix[2][0], matrix[2][1], matrix[2][2])

A = np.array([[1, 1, 2], [0, 1, 1], [0, -3, -2]])
B = np.array([[-2, 0, -1], [-5, 1, -1], [3, 0, 1]])

I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])


# C is the inverse of A
# D is the inverse of B
C = np.array([[1, -4, -1], [0, -2, -1], [0, 3, 1]])
D = np.array([[1, 0, 1], [2, 1, 3], [-3, 0, -2]])

# we are trying to figure out whether A, B, C, and D can generate SL3_Z

In [25]:
def are_we_done_yet(my_matrix):
    return (my_matrix==I).all()

In [26]:
def apply_mat(mat, index):
    if index==0:
        return mat @ A
    elif index==1:
        return mat @ B
    elif index==2:
        return mat @ C
    elif index==3:
        return mat @ D
    assert(1==2)

def generate_successors(last_mat):
    if last_mat==0:
        return [0, 1, 3]
    elif last_mat==1:
        return [0, 1, 2]
    elif last_mat==2:
        return [1, 2, 3]
    elif last_mat==3:
        return [0, 2, 3]
    elif last_mat==4:
        return [0, 1, 2, 3]


In [27]:
# adapted from ChatGPT
visited = set()
start_time = time.time()

def bfs(start):
    
    visited.add(matrix_to_tuple(start @ A))
    visited.add(matrix_to_tuple(start @ B))
    
    start_tuple_A = start @ A
    start_tuple_B = start @ B

    queue = deque([[start_tuple_A, 0, "0"], [start_tuple_B, 1, "1"]])

    while queue:
        vertex = queue.popleft()
        cur_mat = vertex[0]

        if time.time() >= start_time + 1:
            print("Timed out")
            break

        if are_we_done_yet(cur_mat):
            print(vertex)
            print("We found it!")
            break

        for neighbor in [0, 1]:
            # neighbor will take values 0, 1
            new_mat = apply_mat(cur_mat, neighbor)
            new_tuple = matrix_to_tuple(new_mat)
            if not new_tuple in visited:
                visited.add(new_tuple)
                queue.append([new_mat, neighbor, vertex[2] + str(neighbor)])

bfs(A @ B @ A @ B @ A @ B @ A)

# bfs(np.array([[1, 0, 1], [0, -1, -1], [0, 1, 0]]))
# print(visited)
# ran this overnight until it crashed my computer, didn't find anything :(

[array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]]), 1, '1']
We found it!


In [28]:
A @ B @ B @ A @ A @ B @ B @ A

array([[ 11, -48, -14],
       [  6, -25,  -7],
       [-12,  48,  13]])

In [29]:
len(visited)

4

In [30]:
x = np.array([[1, 0, 1], [0, -1, -1], [0, 1, 0]])
y = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
z = np.array([[0, 1, 0], [1, 0, 0], [-1, -1, -1]])

In [31]:
# iterative deepening
# code adapted from https://www.geeksforgeeks.org/iterative-deepening-searchids-iterative-deepening-depth-first-searchiddfs/

# TODO also try this with the visited set. Would need to pass the visited set through the method calls
# visited = set()
 
# A function to perform a Depth-Limited search
# from given source 'src'
def DLS(src,target,maxDepth):
    # print(src)
    if (src[0] == target).all(): 
        print(src)
        return True

    # If reached the maximum depth, stop recursing.
    if maxDepth <= 0: 
        return False

    # Recur for all the vertices adjacent to this vertex
    for i in [0, 1]:
        new_mat = apply_mat(src[0], i)
        if(DLS([new_mat, i, src[2] + str(i)], target, maxDepth-1)):
            return True
    return False

# IDDFS to search if target is reachable from v.
# It uses recursive DLS()
def IDDFS(src, target, maxDepth):

    # Repeatedly depth-limit search till the maximum depth
    i = 0
    while i<maxDepth:
        if (DLS(src, target, i)):
            return True
        i += 1

# target = x
# target = A @ B @ A @ B
# # started with maxDepth=1,000,000
# # now doing maxDepth = 10,000
# maxDepth = 10000
# src = [I, 1, ""]


# if IDDFS(src, target, maxDepth) == True:
#     print("Target is reachable from source within max depth")
# else :
#     print("Target is NOT reachable from source within max depth")

In [41]:
target = x
# target = A @ B @ A @ B
# started with maxDepth=1,000,000
# now doing maxDepth = 1,000
# DLS was crashing with maxDepth=10,000 whne searching for A@B@A@B
# DLS took 1761 minutes before I cut if off with maxDepth=1000
maxDepth = 18
src = [I, 1, ""]


if DLS(src, target, maxDepth) == True:
    print("Target is reachable from source within max depth")
else :
    print("Target is NOT reachable from source within max depth")

Target is NOT reachable from source within max depth


In [61]:
visited = set()

def DLS_w_visited(src,target,maxDepth):
    if (src[0] == target).all(): 
        print(src)
        return True

    # If reached the maximum depth, stop recursing.
    if maxDepth <= 0: 
        return False

    # Recur for all the vertices adjacent to this vertex
    for i in [0, 1]:
        new_mat = apply_mat(src[0], i)
        new_tuple = matrix_to_tuple(new_mat)
        if new_tuple not in visited: 
            visited.add(new_tuple)
            if(DLS_w_visited([new_mat, i, src[2] + str(i)], target, maxDepth-1)):
                return True
    return False

target = x

maxDepth = 70
src = [I, 1, ""]

visited.add(matrix_to_tuple(I))


if DLS_w_visited(src, target, maxDepth) == True:
    print("Target is reachable from source within max depth")
else :
    print("Target is NOT reachable from source within max depth")

len(visited)

Target is NOT reachable from source within max depth


23849627

In [62]:
matrix_to_tuple(y) in visited

False

In [63]:
matrix_to_tuple(z) in visited

False

In [66]:
# Is this infinite index? How about B @ A? 

print(A @ B)
print(A @ B @ A @ B)
print(A @ B@ A @ B@ A @ B)
print(A @ B@ A @ B@ A @ B@ A @ B)

print(B@A)
print(B@A@B@A)
print(B@A@B@A@B@A)
print(B@A@B@A@B@A@B@A)

[[-2  1 -2]
 [-5 -1 -7]
 [ 3  0  4]]
[[ -7  -3 -11]
 [ -6  -4 -11]
 [  6   3  10]]
[[-4 -4 -9]
 [-1 -2 -4]
 [ 3  3  7]]
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [68]:
print(((A @ B@ A @ B@ A @ B@ A @ B)==I).all())
print(((B@A@B@A@B@A@B@A)==I).all())

True
True


In [72]:
X = A @ B @ (A @ A) @ (B @ B)

print(X)
print(X@X)
print(X@X@X)
print(X@X@X@X)
print(X@X@X@X@X)
print(X@X@X@X@X@X)
print(X@X@X@X@X@X@X)
print(X@X@X@X@X@X@X@X)
print(X@X@X@X@X@X@X@X@X)



[[  3   2   5]
 [  7   6  14]
 [-30 -27 -62]]
[[-127 -117 -267]
 [-357 -328 -749]
 [1581 1452 3316]]
[[   6810    6253   14281]
 [  19103   17541   40061]
 [ -84573  -77658 -177359]]
[[ -364229  -334449  -763830]
 [-1021734  -938195 -2142693]
 [ 4523445  4153599  9486181]]
[[  19481070   17888258   40854029]
 [  54648223   50180073  114603566]
 [-241939902 -222158403 -507375611]]
[[-1041959854  -956767095  2109858460]
 [ 1372065496  1611047898 -1834691659]
 [   55437915 -1002590613  1367566954]]
[[-104558995 -366191258  908092240]
 [1714804808 1817551029 1431960142]
 [-634188910  120697354  236017653]]
[[  -55011919 -1154952242 -1821649323]
 [  678271143  1441795732  1072784490]
 [  451719750  1673264265  1065593104]]
[[ -844862017  -804878529  2008586531]
 [ 1418685333 -1777995368    13530543]
 [-1719914831  -648119534 -1727768324]]


In [78]:
np.linalg.eig(X)[0]

array([-5.34857611e+01,  5.21605394e-01, -3.58442705e-02])

In [79]:
np.linalg.eig(A)[0]

array([ 1. +0.j       , -0.5+0.8660254j, -0.5-0.8660254j])

In [74]:
np.linalg.inv(X)

array([[  6., -11.,  -2.],
       [ 14., -36.,  -7.],
       [ -9.,  21.,   4.]])